**Connect google drive**

In [3]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Aug  2 10:50:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
 15573686464772836946600196822111.jpg
 15573686722502623506381280557123.jpg
 2019-05-09-07-52-17-596.jpg
'acc details ticket.gdoc'
'accommodation n registration robocon (1).docx'
'accommodation n registration robocon (1).gdoc'
'accommodation n registration robocon (2).gdoc'
'accommodation n registration robocon.docx'
'accommodation n registration robocon.gdoc'
'advice letter_ neelesh_nmims.gdoc'
 application.mohit.docx
 application.sandeep.docx
'A Quick Guide

**1) Clone the Darknet**



In [5]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14086, done.
remote: Total 14086 (delta 0), reused 0 (delta 0), pack-reused 14086
Receiving objects: 100% (14086/14086), 12.73 MiB | 23.97 MiB/s, done.
Resolving deltas: 100% (9584/9584), done.


**2) Compile Darknet using Nvidia GPU**


In [6]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [7]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
#!mkdir "/mydrive/yolov3"

In [9]:
!echo "number_plate" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/train.txt\nnames = data/obj.names\nbackup = /mydrive/Yolo_V3_for_OpenCV/car_front' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-08-01 11:24:16--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  1.47MB/s    in 1m 55s  

2020-08-01 11:26:12 (1.34 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /mydrive/Yolo_V3_for_OpenCV/car_front/car_front.zip -d data/obj

Archive:  /mydrive/Yolo_V3_for_OpenCV/car_front/car_front.zip
  inflating: data/obj/cf228.txt      
  inflating: data/obj/cf229.jpeg     
  inflating: data/obj/cf229.txt      
  inflating: data/obj/cf230.jpeg     
  inflating: data/obj/cf230.txt      
  inflating: data/obj/cf231.jpeg     
  inflating: data/obj/cf231.txt      
  inflating: data/obj/cf232.jpeg     
  inflating: data/obj/cf232.txt      
  inflating: data/obj/cf233.jpeg     
  inflating: data/obj/cf233.txt      
  inflating: data/obj/cf234.jpeg     
  inflating: data/obj/cf234.txt      
  inflating: data/obj/cf235.jpeg     
  inflating: data/obj/cf235.txt      
  inflating: data/obj/cf236.jpeg     
  inflating: data/obj/cf236.txt      
  inflating: data/obj/cf237.jpeg     
  inflating: data/obj/cf237.txt      
  inflating: data/obj/cf238.jpeg     
  inflating: data/obj/cf238.txt      
  inflating: data/obj/cf239.jpeg     
  inflating: data/obj/cf239.txt      
  inflating: data/obj/4.jpg          
  inflating: data/obj/4.tx

In [11]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.487500 0.738333 0.267500 0.110000

['15', '0.487500', '0.738333', '0.267500', '0.110000']
0 data/obj/cf200.txt
0 0.487500 0.738333 0.267500 0.110000
15 0.486875 0.756667 0.306250 0.106667

['15', '0.486875', '0.756667', '0.306250', '0.106667']
1 data/obj/cf88.txt
0 0.486875 0.756667 0.306250 0.106667
15 0.527500 0.582500 0.145000 0.051667

['15', '0.527500', '0.582500', '0.145000', '0.051667']
2 data/obj/cf204.txt
0 0.527500 0.582500 0.145000 0.051667
15 0.505000 0.753333 0.190000 0.066667

['15', '0.505000', '0.753333', '0.190000', '0.066667']
3 data/obj/cf103.txt
0 0.505000 0.753333 0.190000 0.066667
15 0.474375 0.685000 0.226250 0.083333

['15', '0.474375', '0.685000', '0.226250', '0.083333']
4 data/obj/cf123.txt
0 0.474375 0.685000 0.226250 0.083333
15 0.488750 0.690000 0.215000 0.083333

['15', '0.488750', '0.690000', '0.215000', '0.083333']
5 data/obj/cf87.txt
0 0.488750 0.690000 0.215000 0.083333
15 0.461875 0.600000 0.183750 0.056667

['15', '0.461875', '0.600000', '0.1837

In [12]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/35.jpg', 'data/obj/cf145.jpg', 'data/obj/cf110.jpg', 'data/obj/cf4.jpg', 'data/obj/64.jpg', 'data/obj/cf184.jpg', 'data/obj/cf137.jpg', 'data/obj/cf169.jpg', 'data/obj/cf111.jpg', 'data/obj/cf143.jpg', 'data/obj/cf187.jpg', 'data/obj/cf8.jpg', 'data/obj/cf2.jpg', 'data/obj/cf25.jpg', 'data/obj/cf7.jpg', 'data/obj/cf32.jpg', 'data/obj/7.jpg', 'data/obj/cf6.jpg', 'data/obj/cf170.jpg', 'data/obj/16.jpg', 'data/obj/cf177.jpg', 'data/obj/cf211.jpg', 'data/obj/cf109.jpg', 'data/obj/4.jpg', 'data/obj/cf149.jpg', 'data/obj/cf228.jpg', 'data/obj/cf142.jpg', 'data/obj/cf227.jpg', 'data/obj/cf136.jpg', 'data/obj/cf179.jpg', 'data/obj/cf144.jpg', 'data/obj/49.jpg', 'data/obj/cf173.jpg', 'data/obj/cf106.jpg', 'data/obj/cf196.jpg', 'data/obj/cf9.jpg', 'data/obj/42.jpg', 'data/obj/cf182.jpg', 'data/obj/cf178.jpg', 'data/obj/cf183.jpg', 'data/obj/cf195.jpg', 'data/obj/cf154.jpg', 'data/obj/cf5.jpg', 'data/obj/cf131.jpg', 'data/obj/cf1.jpg', 'data/obj/58.jpg', 'data/obj/cf185.jpg', 'data/obj

In [13]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [14]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/Yolo_V3_for_OpenCV/car_front/yolov3_training_last.weights -dont_show -map

Streaming output truncated to the last 5000 lines.
 total_bbox = 69627, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.850449, GIOU: 0.848915), Class: 0.999894, Obj: 0.997007, No Obj: 0.000690, .5R: 1.000000, .75R: 1.000000, count: 1, class_loss = 0.000002, iou_loss = 0.016883, total_loss = 0.016885 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.862759, GIOU: 0.861930), Class: 0.999029, Obj: 0.993602, No Obj: 0.000759, .5R: 1.000000, .75R: 1.000000, count: 3, class_loss = 0.007716, iou_loss = 0.063527, total_loss = 0.071243 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000002, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 69631, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.791071, GIOU: 0